<a href="https://colab.research.google.com/github/hfnvbh/colab-trt-pose/blob/master/trt-pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch torchvision

In [3]:
!wget -q https://www.dropbox.com/s/p0sfni1dzb359hk/nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427/7fa2af80.pub

Selecting previously unselected package nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427 (1-1) ...
OK


In [4]:
!apt-get update
!apt-get install -y --no-install-recommends libnvinfer5=5.1.2-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-dev=5.1.2-1+cuda10.0
!apt-get install tensorrt
!apt-get install python3-libnvinfer-dev
!apt-get install uff-converter-tf

Get:1 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  InRelease
Ign:1 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  InRelease
Get:2 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release [574 B]
Get:2 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release [574 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release.gpg [836 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release.gpg [836 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-sec

In [5]:
!dpkg -l | grep TensorRT

ii  graphsurgeon-tf                                             5.1.5-1+cuda10.1                                  amd64        GraphSurgeon for TensorRT package
ii  libnvinfer-dev                                              7.0.0-1+cuda10.2                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                                       7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                                          7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer5                                                 5.1.2-1+cuda10.0                                  amd64        TensorRT runtime libraries
ii  libnvinfer7                                                 7.0.0-1+cuda10.2                                  amd64        TensorRT runtime libraries
ii  libnvonnxparsers-dev                               

In [6]:
!pip3 install tqdm cython pycocotools
!apt-get install python3-matplotlib
!git clone https://github.com/NVIDIA-AI-IOT/trt_pose

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-matplotlib-data python3-cycler python3-dateutil python3-olefile
  python3-pil python3-pyparsing python3-six python3-tz ttf-bitstream-vera
Suggested packages:
  apache2 | lighttpd | httpd libjs-jquery-ui-docs python-cycler-doc dvipng
  gir1.2-gtk-3.0 ghostscript inkscape ipython3 python-matplotlib-doc
  python3-cairocffi python3-gi-cairo python3-gobject python3-nose
  python3-pyqt4 python3-scipy python3-sip python3-tornado texlive-extra-utils
  texlive-latex-extra ttf-staypuft python-pil-doc python3-pil-dbg
  python-pyparsing-doc
The following NEW packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-matplotlib-data python3-cycler python3-dateutil python3-matplotlib
  python3-olefile python3-pil python3-pyparsing python3-six python3

In [7]:
!cd trt_pose && python3 setup.py install

running install
running bdist_egg
running egg_info
creating trt_pose.egg-info
writing trt_pose.egg-info/PKG-INFO
writing dependency_links to trt_pose.egg-info/dependency_links.txt
writing top-level names to trt_pose.egg-info/top_level.txt
writing manifest file 'trt_pose.egg-info/SOURCES.txt'
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:304: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'trt_pose.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/parse_objects.py -> build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/train.py -> build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/coco.py -> build/lib.linux-x86_64-

In [0]:
!wget -q https://www.dropbox.com/s/r7ex76ofqsnjkjy/resnet18_baseline_att_224x224_A_epoch_249.pth

In [0]:
import sys
sys.path.append('trt_pose')

In [10]:

import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

ModuleNotFoundError: ignored

In [0]:

import trt_pose.models

In [14]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 9.2MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4538802 sha256=a30607f8c019628ac25dc4e81a7e7e7ea269c4d277ea0d48fde10569f88644e9
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.2-py2.py3-none-any.whl size=62338 sha256=0bf306a3fc8850e30447ef283d142408504707f9e353f220c309038a2673c796
  Stored in directory: /root/.cache/pip/wheels/a7/d6/ac/03a67d071bde6d272d1f7c9ab7f4344fa9d7b9d98bda7fd127
Successfully built pycuda pytools


In [15]:
!apt-get install cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-440 libnvidia-extra-440
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  cuda-11-0 cuda-command-line-tools-11-0 cuda-compiler-11-0 cuda-cudart-11-0
  cuda-cudart-dev-11-0 cuda-cuobjdump-11-0 cuda-cupti-11-0 cuda-cupti-dev-11-0
  cuda-demo-suite-11-0 cuda-documentation-11-0 cuda-driver-dev-11-0
  cuda-drivers cuda-drivers-450 cuda-gdb-11-0 cuda-libraries-11-0
  cuda-libraries-dev-11-0 cuda-memcheck-11-0 cuda-nsight-11-0
  cuda-nsight-compute-11-0 cuda-nsight-systems-11-0 cuda-nvcc-11-0
  cuda-nvdisasm-11-0 cuda-nvml-dev-11-0 cuda-nvprof-11-0 cuda-nvprune-11-0
  cuda-nvrtc-11-0 cuda-nvrtc-dev-11-0 cuda-nvtx-11-0 cuda-nvvp-11-0
  cuda-runtime-11-0 cuda-samples-11-0 cuda-sanitizer-11-0 cuda-toolkit-11-0
  cuda-tools-11-0 cuda-visual-tools-11-0 libcublas-11